# Analyse des schémas de comportement des utilisateurs dans les applications DeFI
## Analyse des API Web 3 en vue de constituer un jeu de données

### API KEYS

In [14]:
from dotenv import load_dotenv
import os 

load_dotenv()
cg_api_key = os.getenv('CG_API_KEY')
eth_api_key = os.getenv('ETH_API_KEY')


### PROTOCOLS

In [ ]:
import requests
import json
import time

KEY_PROTOCOLS = [                                   # Liste des protocoles clés et leurs identifiants sur CoinGecko
    "uniswap",          # DEX
    "curve-dao-token",  # DEX 
    "balancer",         # DEX
    "aave",             # Lending
    "maker",            # Lending
    "yearn-finance",    # Yield Farming
    "harvest-finance",  # Yield Farming
    "dai",              # Stablecoin
    "usd-coin",         # Stablecoin
    "tether",           # Stablecoin
    "nftfi",            # NFT-Fi (optionnel)
]

def get_defi_protocol_info(protocol_id):
    """
    Récupère les informations détaillées d'un protocole spécifique.
    """
    url = f"https://api.coingecko.com/api/v3/coins/{protocol_id}"
    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": cg_api_key                 # Clé API CoinGecko
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            platforms = data.get('platforms', {})

            protocol_info = {
                'name': data.get('name', 'N/A'),
                'type': infer_protocol_type(protocol_id),           # Déduction du type selon la liste fournie
                'blockchain_contracts': [
                    {'blockchain': blockchain, 'contract': contract}
                    for blockchain, contract in platforms.items()
                ],
                'website_url': data.get('links', {}).get('homepage', ['N/A'])[0],           # URL officielle
                'symbol': data.get('symbol', 'N/A'),                                        # Symbole du token associé
                'market_cap_rank': data.get('market_cap_rank', 'N/A'),                      # Classement par capitalisation
                'description': data.get('description', {}).get('en', '')[:250],             # Description limitée à 250 caractères
            }
            return protocol_info
        
        elif response.status_code == 404:
            print(f"Protocole introuvable : {protocol_id}")
        elif response.status_code == 429:
            print("Trop de requêtes. Attente de 10 secondes...")
            time.sleep(10)                                              # Attendre 10 secondes en cas de limite d'API
            return get_defi_protocol_info(protocol_id)                  # Réessayer
        else:
            print(f"Erreur inconnue avec {protocol_id}: {response.status_code}")
    except Exception as e:
        print(f"Erreur avec {protocol_id} : {str(e)}")
    return None

def infer_protocol_type(protocol_id):
    """
    Déduit le type du protocole en fonction de son identifiant.
    """
    if protocol_id in ["uniswap", "sushiswap", "curve-dao-token", "balancer"]:
        return "DEX"
    elif protocol_id in ["aave", "compound", "maker"]:
        return "Lending"
    elif protocol_id in ["yearn-finance", "harvest-finance", "curve-dao-token"]:
        return "Yield Farming"
    elif protocol_id in ["dai", "usd-coin", "tether"]:
        return "Stablecoin"
    elif protocol_id in ["nftfi", "nifty-gateway", "opensea"]:
        return "NFT-Fi"
    else:
        return "DeFi"

def save_protocols_to_json(protocols, filename):
    """
    Sauvegarde une liste de protocoles dans un fichier JSON.
    """
    with open(filename, 'w') as f:
        json.dump(protocols, f, indent=4)


if __name__ == "__main__":                                                  # Exécuter le script                  
    print("Extraction des données pour les protocoles clés...")
    protocols_data = []
    for protocol_id in KEY_PROTOCOLS:
        protocol_info = get_defi_protocol_info(protocol_id)
        if protocol_info:
            protocols_data.append(protocol_info)
                                                                            # Respecter la limite de requêtes : attendre 2 secondes entre chaque requête
            time.sleep(2)

    print(f"{len(protocols_data)} protocoles récupérés.")
    
    save_protocols_to_json(protocols_data, 'key_protocols.json')        # Sauvegarder dans un fichier JSON
    print("Données sauvegardées dans 'key_protocols.json'.")

Extraction des données pour les protocoles clés...
11 protocoles récupérés.
Données sauvegardées dans 'key_protocols.json'.


### CONTRACTS

In [20]:
import json
import pandas as pd

with open('key_protocols.json', 'r') as f:
    protocols_data = json.load(f)

data = []

for protocol in protocols_data:                                         # Parcours des protocoles pour extraire les contrats          
    blockchain_contracts = protocol.get("blockchain_contracts", [])
    
    for contract_info in blockchain_contracts:                          # Parcours des contrats de chaque protocole
        contract_address = contract_info.get("contract")
        blockchain = contract_info.get("blockchain").lower()  
        
        data.append({
            "protocol": protocol.get("name"),
            "blockchain": blockchain,
            "contract": contract_address
        })

df = pd.DataFrame(data)
df.to_csv('contracts_data.csv', index=False)

print("Les données ont été enregistrées dans 'contracts_data.csv'.")

blockchain_counts = df['blockchain'].value_counts()

print("Nombre de contrats associés à chaque blockchain :")
for blockchain, count in blockchain_counts.items():
    print(f"- {blockchain.capitalize()}: {count} contrats")

Les données ont été enregistrées dans 'contracts_data.csv'.
Nombre de contrats associés à chaque blockchain :
- Ethereum: 11 contrats
- Avalanche: 7 contrats
- Energi: 7 contrats
- Polygon-pos: 7 contrats
- Near-protocol: 6 contrats
- Optimistic-ethereum: 6 contrats
- Arbitrum-one: 6 contrats
- Sora: 5 contrats
- Harmony-shard-0: 4 contrats
- Huobi-token: 4 contrats
- Binance-smart-chain: 3 contrats
- Fantom: 3 contrats
- Xdai: 3 contrats
- Base: 3 contrats
- Celo: 2 contrats
- Tron: 2 contrats
- Solana: 2 contrats
- Aptos: 2 contrats
- Algorand: 1 contrats
- Kava: 1 contrats
- Stellar: 1 contrats
- Polygon-zkevm: 1 contrats
- Sui: 1 contrats
- Hedera-hashgraph: 1 contrats
- Polkadot: 1 contrats
- Zksync: 1 contrats
- The-open-network: 1 contrats
